# Energy Meter Examples

## BayLibre's ACME Cape and IIOCapture

More information can be found at https://github.com/ARM-software/lisa/wiki/Energy-Meters-Requirements#iiocapture---baylibre-acme-cape.

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2016-12-08 12:39:45,242 INFO    : root         : Using LISA logging configuration:
2016-12-08 12:39:45,243 INFO    : root         :   /home/vagrant/lisa/logging.conf


#### Import required modules

In [2]:
# Generate plots inline
%matplotlib inline

import os

# Support to access the remote target
import devlib
from env import TestEnv

# RTApp configurator for generation of PERIODIC tasks
from wlgen import RTA, Ramp

## Target Configuration
The target configuration is used to describe and configure your test environment.
You can find more details in **examples/utils/testenv_example.ipynb**.

In [3]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno',
    "host"        : '192.168.0.1',

    # Folder where all the results will be collected
    "results_dir" : "EnergyMeter_IIOCapture",

    # Define devlib modules to load
    "modules" : ["cpufreq"], # Required by rt-app calibration
    "exclude_modules" : [ 'hwmon' ],

    # Energy Meters Configuration for BayLibre's ACME Cape
    "emeter" : {
        "instrument" : "acme",
        "conf" : {
            #'iio-capture' : '/usr/bin/iio-capture',
            #'ip_address' : 'baylibre-acme.local',
        },
        'channel_map' : {
            'Device0' : 0,
            'Device1' : 1,
        }
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'rt-app' ],
    
    # Comment this line to calibrate RTApp in your own platform
    # "rtapp-calib" :  {"0": 360, "1": 142, "2": 138, "3": 352, "4": 352, "5": 353},
}

In [4]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False, force_new=True)
target = te.target

06:32:20  INFO    :         Target - Using base path: /home/derkling/Code/lisa
06:32:20  INFO    :         Target - Loading custom (inline) target configuration
06:32:20  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq']
06:32:20  INFO    :         Target - Connecting linux target:
06:32:20  INFO    :         Target -   username : root
06:32:20  INFO    :         Target -       host : 192.168.0.1
06:32:20  INFO    :         Target -   password : 
06:32:20  INFO    :         Target - Connection settings:
06:32:20  INFO    :         Target -    {'username': 'root', 'host': '192.168.0.1', 'password': ''}
06:32:24  INFO    :         Target - Initializing target workdir:
06:32:24  INFO    :         Target -    /root/devlib-target
06:32:31  INFO    :         Target - Topology:
06:32:31  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
06:32:33  INFO    :       Platform - Loading default EM:
06:32:33  INFO    :       Platform -    /home/derkling/Code/lisa/libs/utils/pla

## Workload Execution and Power Consumptions Samping

Detailed information on RTApp can be found in **examples/wlgen/rtapp_example.ipynb**.

Each **EnergyMeter** derived class has two main methods: **reset** and **report**.
 - The **reset** method will reset the energy meter and start sampling from channels specified in the target configuration. <br>
 - The **report** method will stop capture and will retrieve the energy consumption data. This returns an EnergyReport composed of the measured channels energy and the report file. Each of the samples can also be obtained, as you can see below.

In [5]:
# Create and RTApp RAMP task
rtapp = RTA(te.target, 'ramp', calibration=te.calibration())
rtapp.conf(kind='profile',
           params={
                'ramp' : Ramp(
                    start_pct =  60,
                    end_pct   =  20,
                    delta_pct =   5,
                    time_s    =   0.5).get()
          })

# EnergyMeter Start
te.emeter.reset()

rtapp.run(out_dir=te.res_dir)

# EnergyMeter Stop and samples collection
nrg_report = te.emeter.report(te.res_dir)

06:32:33  INFO    :          WlGen - Setup new workload ramp
06:32:33  INFO    :          RTApp - Workload duration defined by longest task
06:32:33  INFO    :          RTApp - Default policy: SCHED_OTHER
06:32:33  INFO    :          RTApp - ------------------------
06:32:33  INFO    :          RTApp - task [ramp], sched: using default policy
06:32:33  INFO    :          RTApp -  | calibration CPU: 1
06:32:33  INFO    :          RTApp -  | loops count: 1
06:32:33  INFO    :          RTApp - + phase_000001: duration 0.500000 [s] (5 loops)
06:32:33  INFO    :          RTApp - |  period   100000 [us], duty_cycle  60 %
06:32:33  INFO    :          RTApp - |  run_time  60000 [us], sleep_time  40000 [us]
06:32:33  INFO    :          RTApp - + phase_000002: duration 0.500000 [s] (5 loops)
06:32:33  INFO    :          RTApp - |  period   100000 [us], duty_cycle  55 %
06:32:33  INFO    :          RTApp - |  run_time  55000 [us], sleep_time  45000 [us]
06:32:33  INFO    :          RTApp - + phas

In [6]:
logging.info("Collected data:")
!tree $te.res_dir

06:32:42  INFO    : Collected data:


/home/derkling/Code/lisa/results/EnergyMeter_IIOCapture
├── energy.json
├── energy_stats.json
├── output.log
├── ramp_00.json
├── rt-app-ramp-0.log
├── samples_Device0.csv
└── samples_Device1.csv

0 directories, 7 files


## Power Measurements Data

In [7]:
logging.info("Measured channels energy:")
logging.info("%s", nrg_report.channels)

06:32:42  INFO    : Measured channels energy:
06:32:42  INFO    : {'Device1': '324.56', 'Device0': '220.83'}


In [8]:
logging.info("Returned energy file:")
logging.info("  %s", nrg_report.report_file)
!cat $nrg_report.report_file

06:32:42  INFO    : Returned energy file:
06:32:42  INFO    :   /home/derkling/Code/lisa/results/EnergyMeter_IIOCapture/energy.json


{
    "Device0": "220.83", 
    "Device1": "324.56"
}

In [9]:
stats_file = nrg_report.report_file.replace('.json', '_stats.json')
logging.info("Complete energy stats:")
logging.info("  %s", stats_file)
!cat $stats_file

06:32:42  INFO    : Complete energy stats:
06:32:42  INFO    :   /home/derkling/Code/lisa/results/EnergyMeter_IIOCapture/energy_stats.json


{
    "Device0": {
        "cmax": "15.00", 
        "cmin": "00.00", 
        "energy": "220.83", 
        "pavg": "32.78", 
        "pmax": "100.00", 
        "pmin": "00.00", 
        "vmax": "4390.00"
    }, 
    "Device1": {
        "cmax": "22.00", 
        "cmin": "-1.00", 
        "energy": "324.56", 
        "pavg": "57.81", 
        "pmax": "125.00", 
        "pmin": "00.00", 
        "vmax": "4390.00"
    }
}

In [10]:
logging.info("Device0 stats (head)")
samples_file = os.path.join(te.res_dir, 'samples_Device0.csv')
!head $samples_file

06:32:43  INFO    : Device0 stats (head)


"vshunt mV", "vbus mV", "power mW", "current mA", "timestamp ms"
35.0, 4387.5, 25.0, 4.0, 0.0
-42.5, 4386.2, 25.0, -4.0, 10.5
50.0, 4385.0, 25.0, 3.0, 20.7
32.5, 4386.2, 0.0, 9.0, 30.8
92.5, 4387.5, 25.0, 0.0, 41.1
2.5, 4387.5, 25.0, 6.0, 51.2
17.5, 4390.0, 50.0, 2.0, 61.1
35.0, 4386.2, 25.0, 4.0, 71.2
72.5, 4388.8, 50.0, 7.0, 81.1


In [11]:
logging.info("Device1 stats (head)")
samples_file = os.path.join(te.res_dir, 'samples_Device1.csv')
!head $samples_file

06:32:43  INFO    : Device1 stats (head)


"vshunt mV", "vbus mV", "power mW", "current mA", "timestamp ms"
115.0, 4388.8, 50.0, 12.0, 0.0
75.0, 4386.2, 50.0, 18.0, 9.5
177.5, 4387.5, 50.0, 11.0, 19.8
112.5, 4388.8, 75.0, 16.0, 29.7
85.0, 4388.8, 50.0, 9.0, 39.8
182.5, 4388.8, 75.0, 18.0, 49.7
65.0, 4390.0, 50.0, 7.0, 59.8
95.0, 4388.8, 50.0, 14.0, 69.8
135.0, 4390.0, 50.0, 5.0, 79.7
